
<img width="800px" src="../fidle/img/header.svg"></img>

## <!-- TITLE --> [LMNIST2] - Simple classification with CNN
<!-- DESC --> An example of classification using a convolutional neural network for the famous MNIST dataset, using PyTorch Lightning
<!-- AUTHOR : MBOGOL Touye Achille (AI/ML Engineer MIAI/SIMaP) -->

## Objectives :
 - Recognizing handwritten numbers
 - Understanding the principle of a classifier DNN network 
 - Implementation with pytorch lightning 


The [MNIST dataset](http://yann.lecun.com/exdb/mnist/) (Modified National Institute of Standards and Technology) is a must for Deep Learning.  
It consists of 60,000 small images of handwritten numbers for learning and 10,000 for testing.


## What we're going to do :

 - Retrieve data
 - Preparing the data
 - Create a model
 - Train the model
 - Evaluate the result


## Step 1 - Init python stuff

In [ ]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import lightning.pytorch as pl
import torch.nn.functional as F
import torchvision.transforms as T

import sys,os
import multiprocessing
import matplotlib.pyplot as plt

from torchvision import datasets
from torchmetrics.functional import accuracy
from torch.utils.data import Dataset, DataLoader
from modules.progressbar import CustomTrainProgressBar
from lightning.pytorch.loggers import TensorBoardLogger


# Init Fidle environment
import fidle

run_id, run_dir, datasets_dir = fidle.init('LMNIST2')

## Step 2 Retrieve data

MNIST is one of the most famous historic dataset include in torchvision Datasets. `torchvision` provides many built-in datasets in the `torchvision.datasets`.  


In [ ]:
# Load data sets 
train_dataset = datasets.MNIST(root="data", train=True, download=True, transform=None)

test_dataset= datasets.MNIST(root="data", train=False, download=False, transform=None)


In [ ]:
# print info for train data
print(train_dataset)

print()

# print info for test data
print(test_dataset)

In [ ]:
# See the shape of train data and test data
print("x_train : ",train_dataset.data.shape)
print("y_train : ",train_dataset.targets.shape)

print()

print("x_test  : ",test_dataset.data.shape)
print("y_test  : ",test_dataset.targets.shape)

print()

# print number of targets and  values targets
print("Number of Targets :",len(np.unique(train_dataset.targets)))
print("Targets Values    :",    np.unique(train_dataset.targets))

print()

print("Remark that we work with torch tensors and not numpy array, not tensorflow tensor")
print(" -> x_train.dtype = ",train_dataset.data.dtype)
print(" -> y_train.dtype = ",train_dataset.targets.dtype)

## Step 3 -  Preparing your data for training with DataLoaders
The Dataset retrieves our dataset’s features and labels one sample at a time. While training a model, we typically want to pass samples in `minibatches`, reshuffle the data at every epoch to reduce model overfitting, and use Python’s multiprocessing to speed up data retrieval. DataLoader is an iterable that abstracts this complexity for us in an easy API.

In [ ]:
# Before normalization:
x_train=train_dataset.data
print('Before normalization : Min={}, max={}'.format(x_train.min(),x_train.max()))

# After normalization:
## T.Compose creates a pipeline where the provided transformations are run in sequence
transforms = T.Compose(
    [
        # This transforms takes a np.array or a PIL image of integers
        # in the range 0-255 and transforms it to a float tensor in the
        # range 0.0 - 1.0
        T.ToTensor(),

    ]
)


train_dataset = datasets.MNIST(root="data", train=True, download=True, transform=transforms)
test_dataset= datasets.MNIST(root="data", train=False, download=True, transform=transforms)


# print image and label After normalization. 
# iter() followed by next() is used to get some images and label
image,label=next(iter(train_dataset))
print('After normalization  : Min={}, max={}'.format(image.min(),image.max()))


### Have a look

In [ ]:
x_train=train_dataset.data
y_train=train_dataset.targets

In [ ]:
fidle.scrawler.images(x_train, y_train, [27],  x_size=5,y_size=5, colorbar=True, save_as='01-one-digit')
fidle.scrawler.images(x_train, y_train, range(5,41), columns=12, save_as='02-many-digits')

In [ ]:
# train bacth data
train_loader= DataLoader(
  dataset=train_dataset, 
  shuffle=True, 
  batch_size=512,
  num_workers=2
)

# test batch data
test_loader= DataLoader(
  dataset=test_dataset, 
  shuffle=False, 
  batch_size=512,
  num_workers=2 
)

# print image and label  After normalization and batch_size.
image, label=next(iter(train_loader))
print('Shape of first training data batch after use pytorch dataloader :\nbatch images = {} \nbatch labels = {}'.format(image.shape,label.shape))      

## Step 4 - Create Model
About informations about : 
 - [Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers)
 - [Activation](https://www.tensorflow.org/api_docs/python/tf/keras/activations)
 - [Loss](https://www.tensorflow.org/api_docs/python/tf/keras/losses)
 - [Metrics](https://www.tensorflow.org/api_docs/python/tf/keras/metrics)
   
 `Note :` PyTorch provides losses such as the cross-entropy loss (`nn.CrossEntropyLoss`) usually use for classification problem. we're using the softmax function to predict class probabilities. With a softmax output, you want to use cross-entropy as the loss. To actually calculate the loss, we need to pass in the raw output of our network into the loss, not the output of the softmax function. This raw output is usually called the *logits* or *scores*. because in pytorch the cross entropy contain softmax function already.

In [ ]:
class MyNet(nn.Module):
    
    def __init__(self,num_class=10):
        super().__init__()
        self.num_class=num_class
        self.model = nn.Sequential(
            
            # first convolution  
            nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d((2,2)),  
            nn.Dropout2d(0.1),            # Combat overfitting
            
            # second convolution  
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d((2,2)),   
            nn.Dropout2d(0.1),            # Combat overfitting
            
            nn.Flatten(),                 # convert feature map into feature vectors
            
            # MLP network   
            nn.Linear(16*5*5,100),
            nn.ReLU(),
            nn.Dropout1d(0.1),            # Combat overfitting
        
            nn.Linear(100, num_class),    # logits outpout
        )
        
    def forward(self, x):
        x=self.model(x)                   # forward pass
        return x

In [ ]:
class LitModel(pl.LightningModule):
    
    def __init__(self, MyNet):
        super().__init__()
        self.MyNet = MyNet

    # forward pass
    def forward(self, x): 
        return self.MyNet(x)
    

     # optimizer
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer 

    
    def training_step(self, batch, batch_idx):
        # defines the train loop
        x, y  = batch
        
        # forward pass 
        y_hat = self.MyNet(x) 
        
        # loss function
        loss= F.cross_entropy(y_hat, y) 

        # metrics accuracy
        acc=accuracy(y_hat, y,task="multiclass",num_classes=10)    
        
        metrics = {"train_loss": loss, "train_acc": acc}
        
        # logs metrics for each training_step
        self.log_dict(metrics,
                      on_step=False ,
                      on_epoch=True, 
                      prog_bar=True, 
                      logger=True)
        
        return loss

        
    def validation_step(self, batch, batch_idx):
        # defines the valid loop.
        x, y = batch
        
        # forward pass
        y_hat= self.MyNet(x) 

        # loss function MSE
        loss = F.cross_entropy(y_hat, y)                                

        # metrics accuracy
        acc=accuracy(y_hat, y, task="multiclass", num_classes=10)        

        
        metrics = {"test_loss": loss, "test_acc": acc}
        
        # logs metrics for each validation_step
        self.log_dict(metrics,
                      on_step  = False,
                      on_epoch = True, 
                      prog_bar = True, 
                      logger   = True
                     ) 
        
        
        return metrics
        
    
    
    def predict_step(self, batch, batch_idx):
        # defnie the predict loop 
        x, y = batch

        # forward pass
        y_hat = self.MyNet(x)
        
        return y_hat


In [ ]:
# print summary model
model=LitModel(MyNet())
print(model) 

## Step 5 - Train Model

In [ ]:
# loggers data
os.makedirs(f'{run_dir}/logs',   mode=0o750, exist_ok=True)
logger= TensorBoardLogger(save_dir=f'{run_dir}/logs',name="CNN_logs")

In [ ]:
# train model
# trainer = pl.Trainer(accelerator='auto',
#                      max_epochs=16,
#                      logger=logger,
#                      num_sanity_val_steps=0,
#                      callbacks=[CustomTrainProgressBar()]
#                     )

trainer = pl.Trainer(accelerator='auto',
                     max_epochs=16,
                     logger=logger,
                     num_sanity_val_steps=0
                    )

trainer.fit(model=model, train_dataloaders=train_loader, val_dataloaders=test_loader)

## Step 6 - Evaluate
### 6.1 - Final loss and accuracy
Note : With a DNN, we had a precision of the order of : 97.7%

In [ ]:
# evaluate your model
score=trainer.validate(model=model,dataloaders=test_loader, verbose=False)

print('x_test / acc      : {:5.4f}'.format(score[0]['test_acc']))
print('x_test / loss     : {:5.4f}'.format(score[0]['test_loss']))


In [ ]:
score=trainer.validate(model=model,dataloaders=test_loader, verbose=False)

## 6.2 - Plot history
To access logs with tensorboad :
- Under **Docker**, from a terminal launched via the jupyterlab launcher, use the following command:<br>
```tensorboard --logdir <path-to-logs> --host 0.0.0.0```
- If you're **not using Docker**, from a terminal :<br>
```tensorboard --logdir <path-to-logs>```  

**Note:** One tensorboard instance can be used simultaneously.

###  6.3 - Plot results

In [ ]:
# logits outpout by batch size
y_logits= trainer.predict(model=model,dataloaders=test_loader)

# Concat into single tensor
y_logits= torch.cat(y_logits)

# output probabilities values
y_pred_values=F.softmax(y_logits,dim=1)

# Returns the indices of the maximum output probabilities values 
y_pred=torch.argmax(y_pred_values,dim=-1)

In [ ]:
x_test=test_dataset.data
y_test=test_dataset.targets

In [ ]:
fidle.scrawler.images(x_test, y_test, range(0,200), columns=12, x_size=1, y_size=1, y_pred=y_pred, save_as='04-predictions')

### 6.4 - Plot some errors

In [ ]:
errors=[ i for i in range(len(x_test)) if y_pred[i]!=y_test[i] ]
errors=errors[:min(24,len(errors))]
fidle.scrawler.images(x_test, y_test, errors[:15], columns=6, x_size=2, y_size=2, y_pred=y_pred, save_as='05-some-errors')

In [ ]:
fidle.scrawler.confusion_matrix(y_test,y_pred,range(10),normalize=True, save_as='06-confusion-matrix')

In [ ]:
fidle.end()

<div class="todo">
    A few things you can do for fun:
    <ul>
        <li>Changing the network architecture (layers, number of neurons, etc.)</li>
        <li>Display a summary of the network</li>
        <li>Retrieve and display the softmax output of the network, to evaluate its "doubts".</li>
    </ul>
</div>

---
<img width="80px" src="../fidle/img/logo-paysage.svg"></img>